In [0]:
import tensorflow as tf
import meteo_path as me

In [0]:
import altair as alt
import pdvega

In [0]:
z = tf.Variable([0.0, 0.0], dtype=tf.float32, name='z')
eps = tf.Variable([0.01, 0.01], dtype=tf.float32, name='eps')

In [0]:
a = tf.Variable([1.0, 2.0], dtype=tf.float32, name='a')
b = tf.Variable([2.0, 3.0], dtype=tf.float32, name='b')

In [0]:
def make_model():
    eqr = me.rename(a**2-1+z*(7-5*a)-b**2, 'real')
    eqi = me.rename(b*(2*a-5*z)-eps, 'imag')
    return [eqr, eqi]

In [0]:
mod = me.Model([z, eps], [a, b], make_model)

In [0]:
mod.solve_system(output=True)

In [0]:
z0, eps0 = [0.0, 0.0], [0.001, 0.001]
z1, eps1 = [0.1, 0.1], [0.001, 0.001]
z.assign(z0)
eps.assign(eps0)

In [0]:
%lprun -f mod.homotopy_path mod.homotopy_path([z1, eps1], delt=0.1, solve=True, output=True)

In [0]:
np.r_[np.arange(5), 0]